给定两个大小为 m 和 n 的有序数组 nums1 和 nums2。

请你找出这两个有序数组的中位数，并且要求算法的时间复杂度为 O(log(m + n))。

你可以假设 nums1 和 nums2 不会同时为空。

示例 1:

nums1 = [1, 3]

nums2 = [2]

则中位数是 2.0

示例 2:

nums1 = [1, 2]

nums2 = [3, 4]

则中位数是 (2 + 3)/2 = 2.5

In [2]:
''' cr_2022, Hard'''
from typing import List
class Solution1:
    """ 朴素的归并想法，时间(n+m) """
    def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
        L1 = len(nums1)
        L2 = len(nums2)
        i, j, k = 0, 0, 0
        nums3 = [0] * (L1 + L2)
        while i < L1 and j < L2:
            if nums1[i] <= nums2[j]:
                nums3[k] = nums1[i]
                i += 1
            else:
                nums3[k] = nums2[j]
                j += 1
            k += 1
        if i < L1:
            nums3[k:] = nums1[i:]
        else:
            nums3[k:] = nums2[j:]
        L3 = len(nums3)
        m_ind = L3 // 2
        if L3 % 2 == 1:
            median = nums3[m_ind]
        else:
            median = (nums3[m_ind-1] + nums3[m_ind]) / 2
        return float(median)


class Solution2:
    """ 分治，时间log(n+m) """
    def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
        n, m = len(nums1), len(nums2)
        if (n+m)%2 != 0:
            return self.findkthnum((n+m)//2+1, nums1, nums2)
        if (n+m)%2 == 0:
            return (self.findkthnum((n+m)//2, nums1, nums2) + self.findkthnum((n+m)//2+1, nums1, nums2)) / 2

    def findkthnum(self, k, nums1, nums2):
        # 如果一个数组已经是None了就直接返回另一个的第k小的数
        if not nums1:
            return nums2[k-1]
        elif not nums2:
            return nums1[k-1]
        else:
            if k == 1:
                return min(nums1[0], nums2[0])
            else:
                # 索引是k//2 -1， 对越界的情况进行限制
                nums1_p = min(len(nums1)-1, k//2-1)
                nums2_p = min(len(nums2)-1, k//2-1)
                if nums1[nums1_p] < nums2[nums2_p]:
                    # 如果k//2-1的索引越界，则去掉的部分是整个数组，传进递归函数的部分是None
                    return self.findkthnum(k-min(k//2, len(nums1)), nums1[k//2:], nums2)
                else:
                    return self.findkthnum(k-min(k//2, len(nums2)), nums1, nums2[k//2:])

nums1 = [1,3]
nums2 = [2]
print(Solution1().findMedianSortedArrays(nums1, nums2))
print(Solution2().findMedianSortedArrays(nums1, nums2))

2.0
2


#### 分治法
如何想到分治法？
<br />     ................... left_part  |  right_part....................
<br />    A[0], A[1], ..., A[i-1]  |  A[i], A[i+1], ..., A[m-1]
<br />    B[0], B[1], ..., B[j-1]  |  B[j], B[j+1], ..., B[n-1]

1. 用中位数左右个数相等作为第一个条件，推出i，j的关系；
2. i，j是此消彼长的，A，B也是有序（假设小到大），故可以考虑分治法

这里是详细解题思路[leetcode](https://leetcode-cn.com/problems/median-of-two-sorted-arrays/solution/xun-zhao-liang-ge-you-xu-shu-zu-de-zhong-wei-shu-b/)

In [14]:
def findM1(A, B):
    m, n = len(A), len(B)
    if m > n:
        A, B, m, n = B, A, n, m
    if n == 0:
        raise ValueError
    imin, imax, media = 0, m, int((m+n+1)/2)
    # 要仔细考虑奇偶问题
    while imin <= imax:
        i = int((imin + imax) / 2)
        j = media - i
        if i < m and B[j-1] > A[i]:
            imin = i + 1
        elif i > 0 and A[i-1] > B[j]:
            imax = i - 1
        else:
            if i == 0: max_of_left = B[j-1]
            elif j == 0: max_of_left = A[i-1]
            else: max_of_left = max(A[i-1], B[j-1])
                
            if (m+n) % 2 == 1:
                return float(max_of_left)
            
            if i == m: min_of_right = B[j]
            elif j == n: min_of_right = A[i]
            else: min_of_right = min(B[j], A[i])
            return (max_of_left + min_of_right) / 2.0

In [13]:
findM1(nums1, nums2)

3

#### 上面这个方法不好理解（解体中的方法4）后面用方法3重写一下；
需要达到log(m+n)，就需要2分。中位数，其实就是求第k小的特殊情况，由于序列是有序的，可以每次遍历排除一半的数，达到二分的目的。
这里k=(n+m)//2，注意当k大于二者之一时（假设k>n)，接下来应该取m的第k-n个即可

https://leetcode.cn/problems/median-of-two-sorted-arrays/solutions/8999/xiang-xi-tong-su-de-si-lu-fen-xi-duo-jie-fa-by-w-2/

In [1]:
from typing import List
class Solution:
    def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
        n, m = len(nums1), len(nums2)

        def findkthnum(k, nums1, nums2):
            # 如果一个数组已经是None了就直接返回另一个的第k小的数
            if not nums1:
                return nums2[k-1]
            elif not nums2:
                return nums1[k-1]
            else:
                if k == 1:
                    return min(nums1[0], nums2[0])
                else:
                    # 索引是k//2 -1， 对越界的情况进行限制
                    nums1_p = min(len(nums1)-1, k//2-1)
                    nums2_p = min(len(nums2)-1, k//2-1)
                    if nums1[nums1_p] < nums2[nums2_p]:
                        # 如果k//2-1的索引越界，则去掉的部分是整个数组，传进递归函数的部分是None
                        return findkthnum(k-min(k//2, len(nums1)), nums1[k//2:], nums2)
                    else:
                        return findkthnum(k-min(k//2, len(nums2)), nums1, nums2[k//2:])
        
        if (n+m)%2 != 0:
            return findkthnum((n+m)//2+1, nums1, nums2)
        
        if (n+m)%2 == 0:
            return (findkthnum((n+m)//2, nums1, nums2) + findkthnum((n+m)//2+1, nums1, nums2)) /2


nums1 = [2]
nums2 = [3, 4]
print(Solution().findMedianSortedArrays(nums1, nums2))

3
